In [ ]:
# HIDDEN
# For Tables reference see http://data8.org/datascience/tables.html
# This useful nonsense should just go at the top of your notebook.
from datascience import *
%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plots
import numpy as np
plots.style.use('fivethirtyeight')
plots.rc('lines', linewidth=2, color='r')
from ipywidgets import interact
# datascience version number of last run of this notebook
version.__version__

In [ ]:
# The following is a bunch of code to make data presentation simple.  Skip it for reading.
import sys
sys.path.append("..")
from timetable import TimeTable

import locale
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

import power
import datetime
from os import path

In [ ]:
ucb = power.CampusPower('https://campus-export.xbos.io')

In [ ]:
mv = ucb.model('Barrows Hall', 'Building_Power_Demand_Sensor')

In [ ]:
day = "2019-02-02"

In [ ]:
mv.getday(day)

In [ ]:
def all_sites(campus, day):
    sites = []
    for site in campus.sites():
        #print(site)
        try :
            mv = ucb.model(site, 'Building_Power_Demand_Sensor')
            stbl = mv.getday(day)
            if not sites :
                sites = stbl
            else :
                sites.append(stbl)
        except :
            print('Failed:', site)
    return sites

In [ ]:
raw_models = all_sites(ucb, day)

In [ ]:
def estPave(t1, t2, t3, t4, p_lo, p_hi):
    t1 = max(0, t1)
    t2 = max(t1, t2)
    t3 = max(t2, t3)
    t4 = max(t3, t4) if t4 > t1 else 24
    return p_lo + (p_hi - p_lo) * ((t2-t1)/2 + (t3-t2) + (t4-t3)/2)/24

In [ ]:
models = raw_models.where(np.equal(np.isnan(raw_models['t1']), False))

In [ ]:
models['p_ave'] = models.apply(estPave, 't1', 't2', 't3', 't4', 'p_lo', 'p_hi')

In [ ]:
models.sort('p_ave', descending=True)

In [ ]:
v = ucb.view('SRB1 and Oxford Tract', 'Building_Power_Demand_Sensor')

In [ ]:
ts, md = v.getday(day)

In [ ]:
models.num_rows

In [ ]:
ts.plot('hour')

In [ ]:
models['hi/lo'] = models['p_hi']/models['p_lo']

In [ ]:
def basedaily(p_lo):
    return p_lo * 24

def userdaily(t1, t2, t3, t4, p_lo, p_hi):
    return ((t2-t1)/2 + (t3-t2) + (t4-t3)/2)*p_hi

In [ ]:
models['base kwh'] = models.apply(basedaily, 'p_lo')
models['use kwh'] = models.apply(userdaily, 't1', 't2', 't3', 't4', 'p_lo', 'p_hi')
models['use ratio'] = models['use kwh'] / models['base kwh']

In [ ]:
models.sort('use ratio', descending=True).show()

In [ ]:
models.select(['p_lo', 'p_hi']).scatter('p_lo')

In [ ]:
models.select(['t1', 't2', 't3', 't4']).hist(bins=48)

In [ ]:
models.select('hi/lo').hist(bins=20)